### Importing Packages

In [3]:
import pandas as pd
import numpy as np
from math import pow
from functools import partial
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import haversine

### Loading Data

In [4]:
events_data = pd.read_csv("./events_3.csv")
location_data = pd.read_csv("./iata.csv")

In [5]:
events_data.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0


In [6]:
location_data.head()

,iata_code,lat,lon
0,LON,51.50,-0.17
1,MOW,55.75,37.70
2,NYC,40.71,-74.01
3,PAR,48.85,2.35
4,IST,41.01,28.95


### Data Sanity Check

In [7]:
location_data.groupby("iata_code").count()

,lat,lon
iata_code,,
ABV,2,2
ADB,1,1
ADD,2,2
ADL,2,2
AEP,1,1
AER,2,2
AGP,2,2
AKL,2,2
ALA,2,2


In IATA Codes multiple locations are given for many codes. Dropping the duplicates and keeping only the first location for the sake of data sanity.

In [8]:
location_data.drop_duplicates(subset= 'iata_code', inplace=True)

In [9]:
events_data.drop_duplicates(inplace=True)

### Merging DataFrames

In [10]:
events_data[events_data.user_id == 'f953f0']

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0
134,2017-04-27 22:40:42,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0
359,2017-04-27 21:57:56,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0
9688,2017-04-27 21:56:36,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0


In [11]:
events_origin_loc = events_data.merge(location_data, left_on='origin', right_on='iata_code', how = "left")
events_origin_loc.rename(columns={'lat' : 'origin_lat', "lon" : "origin_lon"}, inplace=True)
events_origin_loc.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,iata_code,origin_lat,origin_lon
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,PAR,48.85,2.35
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,FRA,50.12,8.68
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0,BER,52.52,13.41
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0,DEL,28.57,77.10


In [12]:
events_origin_loc[events_origin_loc.user_id == 'f953f0']

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,iata_code,origin_lat,origin_lon
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41
134,2017-04-27 22:40:42,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41
358,2017-04-27 21:57:56,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41
9260,2017-04-27 21:56:36,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41


In [13]:
events_location = events_origin_loc.merge(location_data, left_on='destination', right_on='iata_code', how = "left")
events_location.rename(columns={'lat' : 'dest_lat', "lon" : "dest_lon"}, inplace=True)
events_location.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,iata_code_x,origin_lat,origin_lon,iata_code_y,dest_lat,dest_lon
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,PAR,48.85,2.35,NYC,40.71,-74.01
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,FRA,50.12,8.68,WAS,38.90,-77.04
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41,CGN,50.88,7.12
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0,BER,52.52,13.41,BCN,41.39,2.16
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0,DEL,28.57,77.10,BKK,13.75,100.50


In [12]:
events_location[events_location.user_id == 'f953f0']

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,iata_code_x,origin_lat,origin_lon,iata_code_y,dest_lat,dest_lon
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41,CGN,50.88,7.12
134,2017-04-27 22:40:42,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41,CGN,50.88,7.12
358,2017-04-27 21:57:56,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41,CGN,50.88,7.12
9260,2017-04-27 21:56:36,search,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41,CGN,50.88,7.12


### Calculating Distances

In [14]:
calc_distance = lambda x: haversine.haversine((x[0], x[1]), (x[2], x[3]))

events_location.loc[:,'distance_km'] = events_location[['origin_lat', 'origin_lon', 'dest_lat', 'dest_lon']].apply(calc_distance, axis=1) 

In [15]:
events_location.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,iata_code_x,origin_lat,origin_lon,iata_code_y,dest_lat,dest_lon,distance_km
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,PAR,48.85,2.35,NYC,40.71,-74.01,5837.828026
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,FRA,50.12,8.68,WAS,38.90,-77.04,6530.035012
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,BER,52.52,13.41,CGN,50.88,7.12,470.077408
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0,BER,52.52,13.41,BCN,41.39,2.16,1499.761224
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0,DEL,28.57,77.10,BKK,13.75,100.50,2923.178366


### Data Understanding

In [23]:
events_data.event_type.unique()

array(['search', 'book'], dtype=object)

There are two types of events i.e., Search and Book

In [40]:
events_location.shape,\
events_location[events_location.event_type == 'book'].shape,\
events_location[events_location.event_type == 'search'].shape

((45898, 18), (1808, 18), (44090, 18))

Total Events = 45898
Total Searches = 44,090
Total Bookings = 1,808

In [16]:
events_location.loc[:,'ts_dt'] = pd.to_datetime(events_location.ts)
events_location.ts_dt.max() - events_location.ts_dt.min()

Timedelta('13 days 23:25:25')

The events data is spanned over around 14 days. 

In [63]:
len(events_location.user_id.unique())

31179

There are 31,179 unique users

In [27]:
user_booking_counts = events_location[events_location.event_type=='book'].groupby("user_id").agg({'event_type': 'count'})
user_booking_counts.reset_index(inplace=True)
user_booking_counts[user_booking_counts.event_type > 1]

,user_id,event_type
52,08e914,2
734,690a8c,3


Only 2 users have done booking more than once.

In [28]:
uids = user_booking_counts[user_booking_counts.event_type > 1].user_id
events_location[events_location.event_type == 'book'][events_location.user_id.isin(uids.values)]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,iata_code_x,origin_lat,origin_lon,iata_code_y,dest_lat,dest_lon,distance_km,ts_dt
4625,2017-04-21 02:43:58,book,690a8c,2017-07-12,2017-07-12,BNE,SIN,2,2,BNE,-27.47,153.03,SIN,1.29,103.85,6147.753956,2017-04-21 02:43:58
4767,2017-04-21 02:43:51,book,690a8c,2017-07-12,2017-07-12,BNE,SIN,2,2,BNE,-27.47,153.03,SIN,1.29,103.85,6147.753956,2017-04-21 02:43:51
6555,2017-04-26 00:57:37,book,08e914,2017-04-27,2017-04-28,DUS,VIE,1,0,DUS,51.22,6.78,VIE,48.12,16.56,782.845564,2017-04-26 00:57:37
8438,2017-04-28 18:34:01,book,08e914,2017-05-03,2017-05-03,DUS,VIE,1,0,DUS,51.22,6.78,VIE,48.12,16.56,782.845564,2017-04-28 18:34:01
8712,2017-04-27 23:52:13,book,690a8c,2017-05-10,2017-05-10,MUC,DUS,1,0,MUC,48.14,11.58,DUS,51.22,6.78,486.185022,2017-04-27 23:52:13


In [60]:
events_location.loc[:,'trip'] = events_location[['origin', 'destination']].apply(lambda x : x[0] + "_" + x[1], axis= 1)
events_location.loc[:,'dt_from'] = pd.to_datetime(events_location.date_from)
events_location.loc[:,'dt_to'] = pd.to_datetime(events_location.date_to)
events_location.loc[:,'duration'] = events_location[['dt_to', 'dt_from']].apply(lambda x : x[0] - x[1], axis =1)
events_location.loc[:,'from_to'] = events_location[['date_from', 'date_to']].apply(lambda x : str(x[0]) + "_" + str(x[1]), axis= 1)

In [70]:
len(events_location[['from_to', 'trip', 'num_adults', 'num_children']].drop_duplicates()),\
len(events_location[events_location.event_type == 'book'][['from_to', 'trip', 'num_adults', 'num_children']].drop_duplicates())

(35787, 1801)

In [71]:
1801 / 35787

0.05032553720624808

There are only 35,787 unique search/book criteria, out of which 1801 are booked. There is only 5% success rate for every unique search.

### Creating 'search and book' timeline for each user and trip

In [72]:
event_timeline = events_location.groupby(["user_id", 'trip', 'from_to', 'distance_km', 'duration']).\
agg({'ts_dt':list, 'event_type': list, 'num_adults' : list, 'num_children' : list})


In [73]:
event_timeline.reset_index(inplace=True)
event_timeline.head()

,user_id,trip,from_to,distance_km,duration,ts_dt,event_type,num_adults,num_children
0,000069,FRA_ALG,2017-05-10_2017-05-26,1553.717725,16 days,[2017-04-29 17:59:51],[search],[2],[3]
1,00019d,PAR_SSA,2017-09-15_2017-10-13,7944.570036,28 days,[2017-04-28 19:14:23],[search],[4],[0]
2,0002e3,PAR_ATH,2017-05-01_2017-05-01,2095.453304,0 days,[2017-04-24 21:25:10],[search],[1],[0]
3,000332,NCE_TLV,2017-06-08_2017-07-07,2722.329760,29 days,"[2017-04-27 16:35:35, 2017-04-27 16:34:02, 201...","[search, search, search]","[1, 1, 2]","[0, 0, 0]"
4,000332,TLV_NCE,2017-06-08_2017-07-07,2722.329760,29 days,[2017-04-27 21:24:01],[search],[2],[0]


In [150]:
booking_timelines = event_timeline[event_timeline.event_type.map(lambda x : 'book' in x)]
non_booking_timelines = event_timeline[event_timeline.event_type.map(lambda x : 'book' not in x)]

In [151]:
booking_timelines.loc[:, 'ts_dt_sorted'] = booking_timelines.ts_dt.apply(sorted)
non_booking_timelines.loc[:, 'ts_dt_sorted'] = non_booking_timelines.ts_dt.apply(sorted)

In [133]:
booking_timelines

,user_id,trip,from_to,distance_km,duration,ts_dt,event_type,num_adults,num_children,ts_dt_sorted
31,0035d4,DUS_AGP,2018-02-08_2018-02-20,1839.949396,12 days,"[2017-04-29 03:43:29, 2017-04-28 04:26:21, 201...","[search, search, search, search, search, searc...","[3, 3, 2, 3, 2, 3, 3, 3, 3]","[1, 1, 0, 1, 0, 1, 1, 1, 1]","[2017-04-24 02:20:28, 2017-04-24 02:29:04, 201..."
42,004671,DUS_MUC,2017-05-14_2017-05-15,486.185022,1 days,"[2017-04-24 15:21:38, 2017-04-24 16:09:05, 201...","[search, book, search, search]","[1, 1, 1, 1]","[0, 0, 0, 0]","[2017-04-24 14:53:29, 2017-04-24 15:21:38, 201..."
49,005e14,PAR_YMQ,2017-05-07_2017-05-20,5506.315396,13 days,"[2017-04-24 17:19:40, 2017-04-24 17:32:04, 201...","[search, book, search]","[1, 1, 1]","[0, 0, 0]","[2017-04-24 17:18:30, 2017-04-24 17:19:40, 201..."
57,006e35,HAM_TXL,2017-08-23_2017-08-31,246.123282,8 days,"[2017-04-26 13:56:40, 2017-04-26 14:03:06]","[search, book]","[1, 1]","[0, 0]","[2017-04-26 13:56:40, 2017-04-26 14:03:06]"
165,011d58,BER_FRA,2017-04-28_2017-04-28,423.247353,0 days,"[2017-04-28 14:33:03, 2017-04-28 14:25:46]","[book, search]","[1, 1]","[0, 0]","[2017-04-28 14:25:46, 2017-04-28 14:33:03]"
208,0166cf,MUC_PMI,2017-07-06_2017-07-15,1188.284804,9 days,"[2017-04-29 15:01:47, 2017-04-21 17:45:50, 201...","[search, search, search, book, search]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[2017-04-21 17:45:50, 2017-04-21 18:11:16, 201..."
218,0182d0,DUS_KTM,2017-06-16_2017-07-05,6871.855604,19 days,"[2017-04-19 00:23:07, 2017-04-18 22:54:40, 201...","[book, search, search]","[2, 2, 2]","[0, 0, 0]","[2017-04-18 22:52:30, 2017-04-18 22:54:40, 201..."
287,01f2a2,CGN_BER,2017-05-19_2017-05-21,470.077408,2 days,"[2017-04-29 20:29:09, 2017-04-29 20:14:31]","[book, search]","[2, 2]","[0, 0]","[2017-04-29 20:14:31, 2017-04-29 20:29:09]"
329,022b2f,STR_ANK,2017-08-03_2017-08-17,2112.058125,14 days,"[2017-04-24 12:19:29, 2017-04-24 12:33:45, 201...","[search, book, search]","[2, 2, 2]","[0, 0, 0]","[2017-04-24 12:18:45, 2017-04-24 12:19:29, 201..."
374,027919,BER_AGP,2017-10-12_2017-11-02,2240.628843,21 days,"[2017-04-21 18:25:10, 2017-04-21 19:17:40, 201...","[search, book, search, search]","[2, 2, 2, 2]","[0, 0, 0, 0]","[2017-04-21 18:12:52, 2017-04-21 18:25:10, 201..."


In [152]:
non_booking_timelines

,user_id,trip,from_to,distance_km,duration,ts_dt,event_type,num_adults,num_children,ts_dt_sorted
0,000069,FRA_ALG,2017-05-10_2017-05-26,1553.717725,16 days,[2017-04-29 17:59:51],[search],[2],[3],[2017-04-29 17:59:51]
1,00019d,PAR_SSA,2017-09-15_2017-10-13,7944.570036,28 days,[2017-04-28 19:14:23],[search],[4],[0],[2017-04-28 19:14:23]
2,0002e3,PAR_ATH,2017-05-01_2017-05-01,2095.453304,0 days,[2017-04-24 21:25:10],[search],[1],[0],[2017-04-24 21:25:10]
3,000332,NCE_TLV,2017-06-08_2017-07-07,2722.329760,29 days,"[2017-04-27 16:35:35, 2017-04-27 16:34:02, 201...","[search, search, search]","[1, 1, 2]","[0, 0, 0]","[2017-04-27 16:34:02, 2017-04-27 16:35:35, 201..."
4,000332,TLV_NCE,2017-06-08_2017-07-07,2722.329760,29 days,[2017-04-27 21:24:01],[search],[2],[0],[2017-04-27 21:24:01]
5,0004e1,MUC_ROM,2017-07-19_2017-07-25,698.788332,6 days,"[2017-04-29 01:06:41, 2017-04-29 01:06:18]","[search, search]","[1, 1]","[0, 0]","[2017-04-29 01:06:18, 2017-04-29 01:06:41]"
6,0004f1,LYS_YUL,2017-09-15_2017-10-05,5849.529555,20 days,[2017-04-29 23:42:19],[search],[2],[0],[2017-04-29 23:42:19]
7,0006d6,CDG_TLV,2017-04-30_2017-05-22,3271.590472,22 days,"[2017-04-28 17:35:16, 2017-04-28 17:35:15]","[search, search]","[1, 1]","[0, 0]","[2017-04-28 17:35:15, 2017-04-28 17:35:16]"
8,000a42,BKK_HKT,2017-07-21_2017-07-26,670.747014,5 days,[2017-04-28 00:17:32],[search],[1],[0],[2017-04-28 00:17:32]
9,000a72,LYS_ALG,2017-05-07_2017-06-17,1012.053485,41 days,[2017-04-28 19:46:39],[search],[1],[0],[2017-04-28 19:46:39]


In [155]:
sort_by_key = lambda x : [y for _, y in sorted(zip(x[0], x[1]))]
booking_timelines.loc[:, 'event_type_sorted'] = booking_timelines[['ts_dt', 'event_type']].apply(sort_by_key, axis = 1, raw = True)
booking_timelines.loc[:, 'num_adults_sorted'] = booking_timelines[['ts_dt', 'num_adults']].apply(sort_by_key, axis = 1, raw = True)
booking_timelines.loc[:, 'num_children_sorted'] = booking_timelines[['ts_dt', 'num_children']].apply(sort_by_key, axis = 1, raw = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [157]:
last_is_book = lambda x : 1 if x[-1] == 'book' else 0 
booking_timelines.loc[:,'last_event_book'] = booking_timelines.event_type_sorted.apply(last_is_book)

In [160]:
booking_timelines[booking_timelines.last_event_book == 0]

,user_id,trip,from_to,distance_km,duration,ts_dt,event_type,num_adults,num_children,ts_dt_sorted,event_type_sorted,num_adults_sorted,num_children_sorted,last_event_book
907,062ea8,HAM_VIE,2017-05-04_2017-05-05,760.513960,1 days,"[2017-04-21 18:29:18, 2017-04-21 18:10:51, 201...","[search, search, search, book, search]","[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]","[2017-04-21 15:49:16, 2017-04-21 18:05:14, 201...","[search, search, search, book, search]","[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]",0
2051,0ea441,MUC_ATH,2017-09-16_2017-10-18,1496.045840,32 days,"[2017-04-19 19:31:06, 2017-04-19 18:55:55, 201...","[search, search, search, search, search, book]","[1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0]","[2017-04-19 08:09:20, 2017-04-19 08:14:48, 201...","[search, search, search, search, book, search]","[1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0]",0
2257,102f6f,MUC_IZM,2017-10-17_2017-10-17,1654.949302,0 days,"[2017-04-21 21:13:09, 2017-04-21 21:16:03, 201...","[book, search, search, search, search]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[2017-04-21 21:03:08, 2017-04-21 21:13:09, 201...","[search, book, search, search, search]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]",0
2293,106a19,DUS_MOW,2017-05-25_2017-06-04,2087.872994,10 days,"[2017-04-24 13:50:03, 2017-04-23 13:30:34, 201...","[book, search, search, search]","[1, 1, 1, 1]","[0, 0, 0, 0]","[2017-04-23 13:30:34, 2017-04-24 13:37:13, 201...","[search, search, book, search]","[1, 1, 1, 1]","[0, 0, 0, 0]",0
3008,158b7a,BKK_SGN,2017-09-08_2017-09-08,743.299798,0 days,"[2017-04-18 17:58:01, 2017-04-18 18:06:35, 201...","[search, book, search]","[2, 2, 2]","[0, 0, 0]","[2017-04-18 17:58:01, 2017-04-18 18:06:35, 201...","[search, book, search]","[2, 2, 2]","[0, 0, 0]",0
3409,185fb1,BER_LIS,2017-06-23_2017-07-06,2312.073474,13 days,"[2017-04-21 21:18:36, 2017-04-21 00:39:45, 201...","[search, search, book]","[2, 2, 2]","[0, 0, 0]","[2017-04-21 00:39:45, 2017-04-21 00:59:34, 201...","[search, book, search]","[2, 2, 2]","[0, 0, 0]",0
3599,19bfe4,MUC_SYD,2017-06-01_2017-06-22,16331.855061,21 days,"[2017-04-21 23:19:05, 2017-04-21 23:05:20, 201...","[search, search, search, book, search, search]","[1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0]","[2017-04-21 21:36:34, 2017-04-21 23:01:25, 201...","[search, search, search, search, book, search]","[1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0]",0
3668,1a281d,PAR_CMN,2017-08-17_2017-09-07,1908.325843,21 days,"[2017-04-22 22:31:05, 2017-04-22 22:21:38, 201...","[book, search, search]","[1, 1, 1]","[0, 0, 0]","[2017-04-22 22:21:38, 2017-04-22 22:31:05, 201...","[search, book, search]","[1, 1, 1]","[0, 0, 0]",0
3736,1ab0c1,MUC_DPS,2017-09-02_2017-09-29,11737.924366,27 days,"[2017-04-20 14:08:46, 2017-04-20 14:27:03, 201...","[search, book, search]","[2, 2, 2]","[0, 0, 0]","[2017-04-20 14:08:46, 2017-04-20 14:27:03, 201...","[search, book, search]","[2, 2, 2]","[0, 0, 0]",0
4021,1c9e6a,DUS_PMI,2017-11-19_2017-11-26,1335.099389,7 days,"[2017-04-25 15:59:25, 2017-04-25 15:59:22, 201...","[search, search, book, search, search]","[4, 4, 4, 4, 4]","[1, 1, 1, 1, 1]","[2017-04-25 15:56:53, 2017-04-25 15:59:22, 201...","[search, search, search, book, search]","[4, 4, 4, 4, 4]","[1, 1, 1, 1, 1]",0


In [162]:
booking_timelines.shape, booking_timelines[booking_timelines.last_event_book == 0].shape

((1806, 14), (137, 14))

Out of all 1806 bookings 137 times users search again after booking and rest do not have any more events after search.

In [165]:
from collections import Counter
book_count = lambda x : Counter(x)['book']
booking_timelines.loc[:,'num_book'] = booking_timelines.event_type_sorted.apply(book_count)

In [168]:
booking_timelines[booking_timelines.num_book > 1]

,user_id,trip,from_to,distance_km,duration,ts_dt,event_type,num_adults,num_children,ts_dt_sorted,event_type_sorted,num_adults_sorted,num_children_sorted,last_event_book,num_before,num_book
14958,690a8c,BNE_SIN,2017-07-12_2017-07-12,6147.753956,0 days,"[2017-04-21 02:35:20, 2017-04-21 02:43:58, 201...","[search, book, book]","[2, 2, 2]","[2, 2, 2]","[2017-04-21 02:35:20, 2017-04-21 02:43:51, 201...","[search, book, book]","[2, 2, 2]","[2, 2, 2]",1,2,2


There is only one user who has two book events for same trip.

In [171]:
search_count_before_book = lambda x: len(x[:x.index('book')])
booking_timelines.loc[:,'num_search_before_book'] = booking_timelines.event_type_sorted.apply(search_count_before_book)


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [174]:
booking_timelines

,user_id,trip,from_to,distance_km,duration,ts_dt,event_type,num_adults,num_children,ts_dt_sorted,event_type_sorted,num_adults_sorted,num_children_sorted,last_event_book,num_book,num_search_before_book
31,0035d4,DUS_AGP,2018-02-08_2018-02-20,1839.949396,12 days,"[2017-04-29 03:43:29, 2017-04-28 04:26:21, 201...","[search, search, search, search, search, searc...","[3, 3, 2, 3, 2, 3, 3, 3, 3]","[1, 1, 0, 1, 0, 1, 1, 1, 1]","[2017-04-24 02:20:28, 2017-04-24 02:29:04, 201...","[search, search, search, search, search, searc...","[2, 2, 3, 3, 3, 3, 3, 3, 3]","[0, 0, 1, 1, 1, 1, 1, 1, 1]",1,1,8
42,004671,DUS_MUC,2017-05-14_2017-05-15,486.185022,1 days,"[2017-04-24 15:21:38, 2017-04-24 16:09:05, 201...","[search, book, search, search]","[1, 1, 1, 1]","[0, 0, 0, 0]","[2017-04-24 14:53:29, 2017-04-24 15:21:38, 201...","[search, search, search, book]","[1, 1, 1, 1]","[0, 0, 0, 0]",1,1,3
49,005e14,PAR_YMQ,2017-05-07_2017-05-20,5506.315396,13 days,"[2017-04-24 17:19:40, 2017-04-24 17:32:04, 201...","[search, book, search]","[1, 1, 1]","[0, 0, 0]","[2017-04-24 17:18:30, 2017-04-24 17:19:40, 201...","[search, search, book]","[1, 1, 1]","[0, 0, 0]",1,1,2
57,006e35,HAM_TXL,2017-08-23_2017-08-31,246.123282,8 days,"[2017-04-26 13:56:40, 2017-04-26 14:03:06]","[search, book]","[1, 1]","[0, 0]","[2017-04-26 13:56:40, 2017-04-26 14:03:06]","[search, book]","[1, 1]","[0, 0]",1,1,1
165,011d58,BER_FRA,2017-04-28_2017-04-28,423.247353,0 days,"[2017-04-28 14:33:03, 2017-04-28 14:25:46]","[book, search]","[1, 1]","[0, 0]","[2017-04-28 14:25:46, 2017-04-28 14:33:03]","[search, book]","[1, 1]","[0, 0]",1,1,1
208,0166cf,MUC_PMI,2017-07-06_2017-07-15,1188.284804,9 days,"[2017-04-29 15:01:47, 2017-04-21 17:45:50, 201...","[search, search, search, book, search]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[2017-04-21 17:45:50, 2017-04-21 18:11:16, 201...","[search, search, search, search, book]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]",1,1,4
218,0182d0,DUS_KTM,2017-06-16_2017-07-05,6871.855604,19 days,"[2017-04-19 00:23:07, 2017-04-18 22:54:40, 201...","[book, search, search]","[2, 2, 2]","[0, 0, 0]","[2017-04-18 22:52:30, 2017-04-18 22:54:40, 201...","[search, search, book]","[2, 2, 2]","[0, 0, 0]",1,1,2
287,01f2a2,CGN_BER,2017-05-19_2017-05-21,470.077408,2 days,"[2017-04-29 20:29:09, 2017-04-29 20:14:31]","[book, search]","[2, 2]","[0, 0]","[2017-04-29 20:14:31, 2017-04-29 20:29:09]","[search, book]","[2, 2]","[0, 0]",1,1,1
329,022b2f,STR_ANK,2017-08-03_2017-08-17,2112.058125,14 days,"[2017-04-24 12:19:29, 2017-04-24 12:33:45, 201...","[search, book, search]","[2, 2, 2]","[0, 0, 0]","[2017-04-24 12:18:45, 2017-04-24 12:19:29, 201...","[search, search, book]","[2, 2, 2]","[0, 0, 0]",1,1,2
374,027919,BER_AGP,2017-10-12_2017-11-02,2240.628843,21 days,"[2017-04-21 18:25:10, 2017-04-21 19:17:40, 201...","[search, book, search, search]","[2, 2, 2, 2]","[0, 0, 0, 0]","[2017-04-21 18:12:52, 2017-04-21 18:25:10, 201...","[search, search, search, book]","[2, 2, 2, 2]","[0, 0, 0, 0]",1,1,3
